In [ ]:
# MIC Project Image denoising and inpainting using Deep learning techniques
# Here we are going to use Denoising Auto-encoder as basic form 
# Further we use K stacked denoising auto encoders for initializing our weights
# and then use a deep learning model with 2K-1 hidden layers to finally train our data
# then this will be used for testing

# First, We will create DA model having inputs (xi,yi) and then we will train it and our DA will have output
# layer having h(input)
# then we will predict each xi and yi to get h(xi) and h(yi) and then we will give this to out DA as input again
# as the weights will already be initialized inside the model cause we are not reconstructing it

In [ ]:
!pip install patchify

In [ ]:
#imports
import os
import random
import pathlib
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from tensorflow.keras import layers
from tensorflow.keras import activations



from tensorflow.keras.layers import ZeroPadding2D
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Cropping2D
from tensorflow.keras import activations
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from keras import backend as K
import cv2
from matplotlib import image as matimg, pyplot
import tensorflow as tf
from PIL import Image as im
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Adam
import pickle
import keras.backend as K

In [ ]:
DIM = 32
beta = 0.01
pho = 0.005

In [ ]:
#Adding gaussian iid noise to the images
# barbara
barbara = np.array(im.open('../input/imageinpainting/barbara.pgm'))/256*1.0
barbara = barbara[0:512:2,0:512:2]
pyplot.imshow(barbara,'gray')
pyplot.show()
print(barbara.shape)

#lena
lena = np.array(im.open('../input/imageinpainting/lena.tif'))/256*1.0
lena = lena[0:256:1,0:256:1]
pyplot.imshow(lena,'gray')
pyplot.show()
print(lena.shape)

#mandrill
mandrill = np.array(im.open('../input/imageinpainting/mandrill.pgm'))/256*1.0
mandrill = mandrill[0:256:1,0:256:1]
pyplot.imshow(mandrill,'gray')
pyplot.show()
print(mandrill.shape)

#monalisa
monalisa = np.array(im.open('../input/imageinpainting/monalisa.pgm'))/256*1.0
monalisa = monalisa[0:256:1,0:256:1]
pyplot.imshow(monalisa,'gray')
pyplot.show()
print(monalisa.shape)

#caman
caman = np.array(im.open('../input/imageinpainting/caman.tif'))/256*1.0
caman = caman[0:256:1,0:256:1]
pyplot.imshow(caman,'gray')
pyplot.show()
print(caman.shape)

#einstein
einstein = np.array(im.open('../input/imageinpainting/einstein.pgm'))/256*1.0
einstein = einstein[0:256:1,0:256:1]
pyplot.imshow(einstein,'gray')
pyplot.show()
print(einstein.shape)

noisedbarbara = barbara + np.random.normal(0, .01,(256,256))
pyplot.imshow(noisedbarbara,'gray')
pyplot.show()
noisedlena = lena + np.random.normal(0, .01,(256,256))
pyplot.imshow(noisedlena,'gray')
pyplot.show()
noisedmandrill = mandrill + np.random.normal(0, .01,(256,256))
pyplot.imshow(noisedmandrill,'gray')
pyplot.show()
noisedmonalisa = monalisa + np.random.normal(0, .01,(256,256))
pyplot.imshow(noisedmonalisa,'gray')
pyplot.show()
noisedcaman = caman + np.random.normal(0, .01,(256,256))
pyplot.imshow(noisedcaman,'gray')
pyplot.show()
noisedeinstein = einstein + np.random.normal(0, .01,(256,256))
pyplot.imshow(noisedeinstein,'gray')
pyplot.show()

noisedbarbara = barbara
noisedcaman = caman
noisedeinstein = einstein
noisedmonalisa = monalisa
noisedmandrill = mandrill
noisedlena = lena

images = np.zeros((64*6,32*32))
noiseAddedimages = np.zeros((64*6,32*32))
for i in range(0,8):
    for j in range(0,8):
        images[i*8+j]=barbara[i*32:i*32+32,j*32:j*32+32].reshape(-1)
        noiseAddedimages[i*8+j] = noisedbarbara[i*32:i*32+32,j*32:j*32+32].reshape(-1)
for i in range(0,8):
    for j in range(0,8):
        images[64+i*8+j]=lena[i*32:i*32+32,j*32:j*32+32].reshape(-1)
        noiseAddedimages[64+i*8+j] = noisedlena[i*32:i*32+32,j*32:j*32+32].reshape(-1)
for i in range(0,8):
    for j in range(0,8):
        images[64*2+i*8+j]=mandrill[i*32:i*32+32,j*32:j*32+32].reshape(-1)
        noiseAddedimages[64*2+i*8+j] = noisedmandrill[i*32:i*32+32,j*32:j*32+32].reshape(-1)
for i in range(0,8):
    for j in range(0,8):
        images[64*3+i*8+j]=monalisa[i*32:i*32+32,j*32:j*32+32].reshape(-1)
        noiseAddedimages[64*3+i*8+j] = noisedmonalisa[i*32:i*32+32,j*32:j*32+32].reshape(-1)
for i in range(0,8):
    for j in range(0,8):
        images[64*4+i*8+j]=caman[i*32:i*32+32,j*32:j*32+32].reshape(-1)
        noiseAddedimages[64*4+i*8+j] = noisedcaman[i*32:i*32+32,j*32:j*32+32].reshape(-1)
for i in range(0,8):
    for j in range(0,8):
        images[64*5+i*8+j]=einstein[i*32:i*32+32,j*32:j*32+32].reshape(-1)
        noiseAddedimages[64*5+i*8+j] = noisedeinstein[i*32:i*32+32,j*32:j*32+32].reshape(-1)

# images[0] = barbara.reshape(-1)
# images[1] = lena.reshape(-1)
# images[2] = mandrill.reshape(-1)
# images[3] = monalisa.reshape(-1)
# images[4] = caman.reshape(-1)
# images[5] = einstein.reshape(-1)
# noiseAddedimages = np.zeros((6,32*32))
# noiseAddedimages[0] = barbara.reshape(-1)+np.random.normal(0, .1,(32*32))
# noiseAddedimages[1] = lena.reshape(-1)+np.random.normal(0, .1,(32*32))
# noiseAddedimages[2] = mandrill.reshape(-1)+np.random.normal(0, .1,(32*32))
# noiseAddedimages[3] = monalisa.reshape(-1)+np.random.normal(0, .1,(32*32))
# noiseAddedimages[4] = caman.reshape(-1)+np.random.normal(0, .1,(32*32))
# noiseAddedimages[5] = einstein.reshape(-1)+np.random.normal(0, .1,(32*32))
# pyplot.imshow(noiseAddedimages[0].reshape(32,32),'gray')
# print(np.sum((images[0]-noiseAddedimages[0])**2))

In [ ]:
patches = patchify(barbara,(DIM,DIM),step=16)
print(patches.shape)
patches = patches.reshape(225,DIM,DIM)
images = patches
noiseAddedimages = patchify(noisedbarbara,(DIM,DIM),step=16)
noiseAddedimages = noiseAddedimages.reshape(225,DIM,DIM)
print(images.shape)
print(noiseAddedimages.shape)

In [ ]:
def custom_loss1(y_true, y_pred):
    loss = K.square(y_pred - y_true)
    print(loss.shape)
    loss = K.sum(loss,axis=0)
    print(loss.shape)
    loss = loss*0.5
    loss = K.mean(loss)
    print(loss.shape)
    return loss

In [ ]:
def sparsity_loss(y):
    print(y.shape)
    y = y*1.0
    val = K.mean(y,axis=0)
    print(val)
    print(val.shape)
#     const = tf.constant(pho,shape=val.shape,dtype=tf.float32)
#     a1 = tf.divide(const,val);
#     print(a1.shape)
#     const1 = tf.constant(-1,shape=val.shape,dtype=tf.float32)
#     a2 = tf.add(const1,val)
#     a2 = tf.multiply(const1,a2)
#     print(a2.shape)
#     const2 = tf.constant(1-pho,shape=val.shape,dtype=tf.float32)
#     a2 = tf.divide(const2,a2)
#     print(a2.shape)
#     loss = tf.multiply(const,K.log(a1)) + tf.multiply(const2,K.log(a2))
    a1 = pho/val;
    a2 = 1-val;
    a2 = (1-pho)/a2
    loss = pho*K.log(a1) + (1-pho)*K.log(a2)
    print(loss.shape)
    loss = beta*K.sum(loss)
    print(loss.shape)
    return loss;

In [ ]:
activation = "sigmoid"
kernel_initializer = "random_normal"

inputs = Input((256,256))
s  = Lambda(lambda x: x) (inputs)
layer1 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s1 = layer1(s)
layer2 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s2 = layer2(s1)

model = Model(inputs = inputs, outputs =[s2])

sparsity = sparsity_loss(s1)
model.add_loss(sparsity)
# mse = custom_loss1()

model.compile(optimizer = Adam(0.0001),loss="mse")

In [ ]:
model.summary()

In [ ]:
model.fit(noiseAddedimages,images,epochs=40)

In [ ]:
layer1weights = layer1.get_weights()
activation = "sigmoid"
kernel_initializer = "random_normal"

inputs = Input((32,32))
s  = Lambda(lambda x: x) (inputs)
layer3 = Dense(32,activation=activation)
s1 = layer3(s)
layer3.set_weights(layer1weights)

model1 = Model(inputs = inputs,outputs=[s1])

In [ ]:
X= model.predict(noiseAddedimages[0].reshape(1,32,32))
print(noiseAddedimages[0].reshape((1024)).shape)
print(X.shape)
pyplot.imshow(X.reshape((DIM,DIM)),'gray')
pyplot.show()
pyplot.imshow(noiseAddedimages[0],'gray')

In [ ]:
images1 = images
images1 = model1.predict(images)
    
noiseAddedimages1 = noiseAddedimages
noiseAddedimages1= model1.predict(noiseAddedimages)

layer1weights = layer1.get_weights()
layer2weights = layer2.get_weights()

activation = "sigmoid"
kernel_initializer = "random_normal"

inputs = Input((32,32))
s  = Lambda(lambda x: x) (inputs)
layer1 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s1 = layer1(s)
layer2 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s2 = layer2(s1)

model2 = Model(inputs = inputs, outputs =[s2])

sparsity = sparsity_loss(s1)
model2.add_loss(sparsity)

model2.compile(optimizer = Adam(0.0001),loss="mse")
model2.fit(noiseAddedimages1,images1,epochs=40)

layer3weights = layer1.get_weights()
layer4weights = layer2.get_weights()

In [ ]:
inputs = Input((32,32))
s  = Lambda(lambda x: x) (inputs)
layer1 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s1 = layer1(s)
layer2 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s2 = layer2(s1)
layer3 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
s3 = layer3(s2)
layer4 = Dense(32,activation=activation,kernel_regularizer=tf.keras.regularizers.l2(0.0001/2))
output = layer4(s3)

layer1.set_weights(layer1weights)
layer2.set_weights(layer3weights)
layer3.set_weights(layer4weights)
layer4.set_weights(layer2weights)

modelFinal = model1 = Model(inputs = inputs,outputs=[output])
modelFinal.compile(optimizer = Adam(0.0001),loss = "mse")
modelFinal.fit(noiseAddedimages,images,epochs=40)

In [ ]:
X= modelFinal.predict(noiseAddedimages[100].reshape(1,32,32))
print(noiseAddedimages[0].reshape((1024)).shape)
print(X.shape)
pyplot.imshow(X.reshape((32,32)),'gray')
pyplot.show()
pyplot.imshow(noiseAddedimages[100].reshape((32,32)),'gray')
pyplot.show()
pyplot.imshow(images[100].reshape((32,32)),'gray')